# PPE needs estimation from confirmed cases at county level


![convid19_patient_flow](https://user-images.githubusercontent.com/14057932/77640828-695cee00-6f31-11ea-812b-55c33f2e9de4.png)


## Input data

Input data needs to be divided into two groups:

- patients group a who go to community testing;
- patients group b who go to ED presentation;

Ideally, the patients can be daily tracking by each state or hospital. It can be the simulation number that comes from a predictive model, e.g. [SEIR model](https://github.com/alsnhll/SEIR_COVID19/blob/master/SEIR_COVID19.ipynb)

In [1]:
import os
import os.path as op
from IPython.display import Image
import numpy as np
import pandas as pd
import geopandas as gpd

In [2]:
cases_file = "../../data/external/covid_confirmed_usafacts.csv"
pop_geo = "../../data/processed/us_counties_with_pop.geojson"

In [4]:
# setting at one, so we can read in confirmed case daily from multiple sources
# in this workthough, we used data from https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/

confirmed_case = 1

# one health provider testing 10 patient perday
testing_ratio = 10

positive_rate = 0.14

inpatient_rate = 0.2
homecare_rate = 1- inpatient_rate
test_cases = round(confirmed_case / positive_rate)


critical_rate = 0.2
support_care = 0.6

inpatient_care_days = 7
critical_care_days = 10

## patient to lab technician and healthcare workers
patient_tech_ratio = 175 


# PPE use per patients per day from in patient to critical care
haveay_ppe_use_inp = 10
median_ppe_use_inp = 4 

haveay_ppe_use_icu = 40
median_ppe_use_icu = 6 
light_ppe_use = 1
double = 2


In [5]:
def ppes_ed_community(patients):
    """compute PPEs include (N95, gloves, gown, testkit, face shield and ventilator) 
    base on patients and healthcareworkers’ cosupution in ED or community testing 
    
    Args:
        patients (int): number of comfirmed cases

    Returns:
        total_ppes_ed_comm (dictionary): PPE needs for ED presentation or community testing
    """
    test_cases = round(confirmed_case / positive_rate)
    healthcareworkers = test_cases/testing_ratio
    droplet_masks = test_cases 
    n95_needs =  healthcareworkers
    gloves = healthcareworkers
    testkits =  test_cases
    gowns =  healthcareworkers
    faceshields = healthcareworkers
    total_ppes_ed_comm = dict(N95s= n95_needs, Droplet_Masks = droplet_masks, Gloves =gloves, Testkits = testkits, Gowns=gowns,
                         Faceshields=faceshields)
    
    return total_ppes_ed_comm


def ppes_lab_processing(patients): 
    """to compute the ppes needs during the lab processing
    Args:
        patients (int): number of total comfirmed cases from community test and ED presentation;
        
    Returns:
        total_ppes_lab (dictionary): PPE needs for lab processing
    """
    test_cases = round(confirmed_case / positive_rate)
    technicians = test_cases/patient_tech_ratio # each technician can run 175 test per day. 
    n95_needs = technicians
    gloves =  technicians
    gowns =  technicians
    faceshields = technicians

    total_ppes_lab = dict(N95s=n95_needs, Gloves= gloves, Gowns=gowns, 
                          Faceshields= faceshields)

    return total_ppes_lab


def ppes_home_care(patients):
    """to compute ppe needs during the homecare
    Args:
        patients (int): number of total patients from community test and ED presentation;
        
    Returns:
        total_ppes_homecare (dictionary): PPE needs for home care
    """

    #20% endup being positive and 80% of these in home care
    home_care_patients = patients * homecare_rate        
    droplet_masks_needs = home_care_patients

    total_ppes_homecare = dict(Droplet_Masks = droplet_masks_needs)
    return total_ppes_homecare


def ppes_in_patient_care(patients, inpatient_care_days):
    """to compute ppe needs during in patient care
    Args:
        patients (int): number of total patients from community test and ED presentation;
        in_patients_days(int): number of days each person will need to be in Patient Care
        
    Returns:
        total_ppes_in_patient (dictionary): PPE needs for in patient care
    """

    # 20% endup being positive and 20% of these in patient care
    in_patient_patients = patients * inpatient_rate
    in_patient_days = in_patient_patients * inpatient_care_days
    
    n95_needs =  in_patient_days* haveay_ppe_use_inp
    gloves = in_patient_days * haveay_ppe_use_inp * double # 2x of 40 per patients 
    gowns = in_patient_days * haveay_ppe_use_inp # 40 gowns per paitient
    faceshields = in_patient_days * haveay_ppe_use_inp 
    goggles =  in_patient_days * median_ppe_use_inp # goggle can be reused
    bp_cuff = in_patient_days * light_ppe_use # bp cuff, ambu bag can be reused
    wipers = in_patient_days * light_ppe_use

    total_ppes_in_patient = dict(N95s = n95_needs, Gloves= gloves, Gowns=gowns, 
                                 Faceshields= faceshields, Goggles = goggles, 
                                 BP_cuff = bp_cuff, Wiper = wipers)

    return total_ppes_in_patient


def ppes_critical_care(patients, critical_care_days):
    """compute the ventilators and ppe needs based on the critical care patients
    Args:
        patients (int): number of total patients from community test and ED presentation;
        in_patients_days(int): number of days each person will need to be in Critical Care
        
    Returns:
        total_ppes_criticalcare (dictionary): PPE needs for in critical care
    """

    # 40% of in patient patients end up in critical care
    critical_care_patients = patients * positive_rate * critical_rate
    
    critical_patient_days = critical_care_patients * critical_care_days 
    
    ventilators = critical_care_patients

    # assuming 1 doctor takes care the 3 patients over 10 days- needs to check it with Abi 
    n95_needs = critical_patient_days * haveay_ppe_use_icu
    gloves =  critical_patient_days * haveay_ppe_use_icu * double # 2x of 40 per patients 
    gowns = critical_patient_days * haveay_ppe_use_icu # 40 gowns per paitient
    faceshields = critical_patient_days* haveay_ppe_use_icu
    goggles =  critical_patient_days* median_ppe_use_icu # goggle can be reused
    bp_cuff = critical_patient_days * light_ppe_use # bp cuff, ambu bag can be reused
    wipers = critical_patient_days * light_ppe_use
    ventilators = critical_patient_days * light_ppe_use

    total_ppes_criticalcare = dict(N95s = n95_needs, Gloves= gloves, Gowns=gowns, 
                                  Faceshields= faceshields,Goggles = goggles, 
                                  BP_cuff = bp_cuff, Wiper = wipers,
                                  Ventilators = ventilators)
    
    return total_ppes_criticalcare


In [6]:
# for every confirmed convid19 case
total_ppes_ed_comm = ppes_ed_community(confirmed_case)
total_ppes_ed_comm 

{'N95s': 0.7,
 'Droplet_Masks': 7,
 'Gloves': 0.7,
 'Testkits': 7,
 'Gowns': 0.7,
 'Faceshields': 0.7}

In [7]:
# for every confirmed convid19 case
total_ppes_lab = ppes_lab_processing(confirmed_case)
total_ppes_lab

{'N95s': 0.04, 'Gloves': 0.04, 'Gowns': 0.04, 'Faceshields': 0.04}

In [8]:
# for every confirmed convid19 case
total_ppes_homecare = ppes_home_care(confirmed_case)
total_ppes_homecare

{'Droplet_Masks': 0.8}

In [9]:
# for every confirmed convid19 case
total_ppes_in_patient = ppes_in_patient_care(confirmed_case, inpatient_care_days)
total_ppes_in_patient

{'N95s': 14.000000000000002,
 'Gloves': 28.000000000000004,
 'Gowns': 14.000000000000002,
 'Faceshields': 14.000000000000002,
 'Goggles': 5.6000000000000005,
 'BP_cuff': 1.4000000000000001,
 'Wiper': 1.4000000000000001}

In [10]:
# for every confirmed convid19 case
total_ppes_criticalcare = ppes_critical_care(confirmed_case, critical_care_days)
total_ppes_criticalcare

{'N95s': 11.200000000000001,
 'Gloves': 22.400000000000002,
 'Gowns': 11.200000000000001,
 'Faceshields': 11.200000000000001,
 'Goggles': 1.6800000000000002,
 'BP_cuff': 0.28,
 'Wiper': 0.28,
 'Ventilators': 0.28}

In [11]:
# for every confirmed convid19 case from home in patient and critical cares 
testkits = total_ppes_ed_comm['Testkits']
droplet_masks = total_ppes_homecare['Droplet_Masks'] + total_ppes_ed_comm['Droplet_Masks'] 
n95s =  total_ppes_in_patient['N95s'] + total_ppes_criticalcare['N95s'] + total_ppes_ed_comm['N95s'] + total_ppes_lab['N95s'] + total_ppes_ed_comm['N95s']
gloves = total_ppes_in_patient['Gloves'] + total_ppes_criticalcare['Gloves']  + total_ppes_lab['Gloves']+ total_ppes_ed_comm['Gloves']
gowns = total_ppes_in_patient['Gowns'] + total_ppes_criticalcare['Gowns'] + total_ppes_lab['Gowns']+ total_ppes_ed_comm['Gowns']
faceshields = total_ppes_in_patient['Faceshields'] + total_ppes_criticalcare['Faceshields'] + total_ppes_lab['Faceshields'] + total_ppes_ed_comm['Faceshields']
goggles = total_ppes_in_patient['Goggles'] + total_ppes_criticalcare['Goggles']
bp_cuffs = total_ppes_in_patient['BP_cuff'] + total_ppes_criticalcare['BP_cuff']
wipers = total_ppes_in_patient['Wiper'] + total_ppes_criticalcare['Wiper']
ventilators = total_ppes_criticalcare['Ventilators']

In [12]:
total_ppes = round(sum([testkits, n95s, gloves, gowns, faceshields,
                        droplet_masks, goggles, bp_cuffs, wipers, ventilators]))

In [13]:
# every confirm case may end up using 162 ppes, including n95 masks, gowns, faceshieldd, droplet_masks, googles, bp_cuff, wipers, ventilators
total_ppes

155

In [14]:
confirmed_cases = pd.read_csv(cases_file)
confirmed_cases.head(4)

,countyFIPS,County Name,State,stateFIPS,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,...,3/15/2020,3/16/2020,3/17/2020,3/18/2020,3/19/2020,3/20/2020,3/21/2020,3/22/2020,3/23/2020,3/24/2020
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,1,1,1,1,1,2,2,3,3,4
3,1015,Calhoun County,AL,1,0,0,0,0,0,0,...,0,0,0,1,1,1,1,2,2,2


In [15]:
confirmed_cases.keys()

Index(['countyFIPS', 'County Name', 'State', 'stateFIPS', '1/22/2020',
       '1/23/2020', '1/24/2020', '1/25/2020', '1/26/2020', '1/27/2020',
       '1/28/2020', '1/29/2020', '1/30/2020', '1/31/2020', '2/1/2020',
       '2/2/2020', '2/3/2020', '2/4/2020', '2/5/2020', '2/6/2020', '2/7/2020',
       '2/8/2020', '2/9/2020', '2/10/2020', '2/11/2020', '2/12/2020',
       '2/13/2020', '2/14/2020', '2/15/2020', '2/16/2020', '2/17/2020',
       '2/18/2020', '2/19/2020', '2/20/2020', '2/21/2020', '2/22/2020',
       '2/23/2020', '2/24/2020', '2/25/2020', '2/26/2020', '2/27/2020',
       '2/28/2020', '2/29/2020', '3/1/2020', '3/2/2020', '3/3/2020',
       '3/4/2020', '3/5/2020', '3/6/2020', '3/7/2020', '3/8/2020', '3/9/2020',
       '3/10/2020', '3/11/2020', '3/12/2020', '3/13/2020', '3/14/2020',
       '3/15/2020', '3/16/2020', '3/17/2020', '3/18/2020', '3/19/2020',
       '3/20/2020', '3/21/2020', '3/22/2020', '3/23/2020', '3/24/2020'],
      dtype='object')

In [16]:
new_df = confirmed_cases

In [17]:
new_df.iloc[:, 5:] = np.array(new_df.iloc[:, -62:]) - np.array(new_df.iloc[:, -63:-1])

In [18]:
new_df.head()

,countyFIPS,County Name,State,stateFIPS,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,...,3/15/2020,3/16/2020,3/17/2020,3/18/2020,3/19/2020,3/20/2020,3/21/2020,3/22/2020,3/23/2020,3/24/2020
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1
3,1015,Calhoun County,AL,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
4,1017,Chambers County,AL,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,3


In [20]:
new_df.iloc[:, 4:] = new_df.iloc[:, 4:].clip(lower = 0)

In [21]:
new_df.head()

,countyFIPS,County Name,State,stateFIPS,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,...,3/15/2020,3/16/2020,3/17/2020,3/18/2020,3/19/2020,3/20/2020,3/21/2020,3/22/2020,3/23/2020,3/24/2020
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1
3,1015,Calhoun County,AL,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
4,1017,Chambers County,AL,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,3


In [24]:
def multiple_ppe(ppe_item_total, df, opp_col, item_name):
    """to compute totoal ppes from given dates
    
    Args:
        ppe_item_total (int): the total ppe item from confirm covid19 cases;
        df (Pandas dataframe): data frame that records growning confirm cases 
        opp_col (int): the column where cofirmed case starts to be recorded
        
    Returns:
        ppes_df(Pandas dataframe): dataframe has the total sum of the ppe items
    """
    
    df_n = df.apply(lambda x: x[opp_col:] * ppe_item_total, axis = 1) 
    total_g_ppes = df_n.apply(np.sum, axis =1)
    ppes_df = df.iloc[:,:opp_col]
    
    #if we see patient in the critical care, we assume at least 1 ventilator is needed
    if ppe_item_total == ventilators:
        total_g_ppes = np.floor(total_g_ppes)
    ppes_df[item_name] = round(total_g_ppes)
    return ppes_df
    

In [26]:
total_ppes_df = multiple_ppe(total_ppes, confirmed_cases, 4, "total_ppes")
total_ppes_df.head()

,countyFIPS,County Name,State,stateFIPS,total_ppes
0,0,Statewide Unallocated,AL,1,0
1,1001,Autauga County,AL,1,155
2,1003,Baldwin County,AL,1,620
3,1015,Calhoun County,AL,1,310
4,1017,Chambers County,AL,1,775


In [27]:
total_testkits = multiple_ppe(testkits, confirmed_cases, 4, "testkits")
total_testkits.head()

,countyFIPS,County Name,State,stateFIPS,testkits
0,0,Statewide Unallocated,AL,1,0
1,1001,Autauga County,AL,1,7
2,1003,Baldwin County,AL,1,28
3,1015,Calhoun County,AL,1,14
4,1017,Chambers County,AL,1,35


In [28]:
total_droplet_masks = multiple_ppe(droplet_masks, confirmed_cases, 4, "total_droplet_masks") 
total_droplet_masks.head()

,countyFIPS,County Name,State,stateFIPS,total_droplet_masks
0,0,Statewide Unallocated,AL,1,0.0
1,1001,Autauga County,AL,1,8.0
2,1003,Baldwin County,AL,1,31.0
3,1015,Calhoun County,AL,1,16.0
4,1017,Chambers County,AL,1,39.0


In [29]:
total_n95s = multiple_ppe(n95s, confirmed_cases, 4, "total_n95s") 
total_n95s.head()

,countyFIPS,County Name,State,stateFIPS,total_n95s
0,0,Statewide Unallocated,AL,1,0.0
1,1001,Autauga County,AL,1,27.0
2,1003,Baldwin County,AL,1,107.0
3,1015,Calhoun County,AL,1,53.0
4,1017,Chambers County,AL,1,133.0


In [30]:
total_gloves = multiple_ppe(gloves, confirmed_cases, 4, "total_gloves") 
total_gloves.head()

,countyFIPS,County Name,State,stateFIPS,total_gloves
0,0,Statewide Unallocated,AL,1,0.0
1,1001,Autauga County,AL,1,51.0
2,1003,Baldwin County,AL,1,205.0
3,1015,Calhoun County,AL,1,102.0
4,1017,Chambers County,AL,1,256.0


In [31]:
total_gowns = multiple_ppe(gowns, confirmed_cases, 4, "total_gowns") 
total_gowns.head()

,countyFIPS,County Name,State,stateFIPS,total_gowns
0,0,Statewide Unallocated,AL,1,0.0
1,1001,Autauga County,AL,1,26.0
2,1003,Baldwin County,AL,1,104.0
3,1015,Calhoun County,AL,1,52.0
4,1017,Chambers County,AL,1,130.0


In [32]:
total_faceshields = multiple_ppe(faceshields, confirmed_cases, 4, "total_faceshields") 
total_faceshields.head()


,countyFIPS,County Name,State,stateFIPS,total_faceshields
0,0,Statewide Unallocated,AL,1,0.0
1,1001,Autauga County,AL,1,26.0
2,1003,Baldwin County,AL,1,104.0
3,1015,Calhoun County,AL,1,52.0
4,1017,Chambers County,AL,1,130.0


In [33]:
total_goggles = multiple_ppe(goggles, confirmed_cases, 4, "total_goggles") 
total_goggles.head()

,countyFIPS,County Name,State,stateFIPS,total_goggles
0,0,Statewide Unallocated,AL,1,0.0
1,1001,Autauga County,AL,1,7.0
2,1003,Baldwin County,AL,1,29.0
3,1015,Calhoun County,AL,1,15.0
4,1017,Chambers County,AL,1,36.0


In [34]:
total_bp_cuffs = multiple_ppe(bp_cuffs, confirmed_cases, 4, "total_bp_cuffs") 
total_bp_cuffs.head()

,countyFIPS,County Name,State,stateFIPS,total_bp_cuffs
0,0,Statewide Unallocated,AL,1,0.0
1,1001,Autauga County,AL,1,2.0
2,1003,Baldwin County,AL,1,7.0
3,1015,Calhoun County,AL,1,3.0
4,1017,Chambers County,AL,1,8.0


In [35]:
total_wipers = multiple_ppe(wipers, confirmed_cases, 4, "total_wipers") 
total_wipers.head()

,countyFIPS,County Name,State,stateFIPS,total_wipers
0,0,Statewide Unallocated,AL,1,0.0
1,1001,Autauga County,AL,1,2.0
2,1003,Baldwin County,AL,1,7.0
3,1015,Calhoun County,AL,1,3.0
4,1017,Chambers County,AL,1,8.0


In [36]:
total_ventilators = multiple_ppe(ventilators, confirmed_cases, 4, "total_ventilators") 
total_ventilators.head()

,countyFIPS,County Name,State,stateFIPS,total_ventilators
0,0,Statewide Unallocated,AL,1,0.0
1,1001,Autauga County,AL,1,0.0
2,1003,Baldwin County,AL,1,1.0
3,1015,Calhoun County,AL,1,0.0
4,1017,Chambers County,AL,1,1.0


In [37]:
total_ppes_df['testkits'] =total_testkits['testkits']
total_ppes_df['total_droplet_masks'] = total_droplet_masks['total_droplet_masks']
total_ppes_df['total_n95s'] = total_n95s['total_n95s']
total_ppes_df['total_gowns'] = total_gowns['total_gowns']
total_ppes_df['total_faceshields'] = total_faceshields['total_faceshields']
total_ppes_df['total_goggles'] = total_goggles['total_goggles']
total_ppes_df['total_bp_cuffs'] = total_bp_cuffs['total_bp_cuffs']
total_ppes_df['total_wipers'] = total_wipers['total_wipers']
total_ppes_df['total_ventilators'] = total_ventilators['total_ventilators']

In [38]:
total_ppes_df = total_ppes_df.rename(columns={'countyFIPS': 'COUNTY_FIPS'})
total_ppes_df.head()


,COUNTY_FIPS,County Name,State,stateFIPS,total_ppes,testkits,total_droplet_masks,total_n95s,total_gowns,total_faceshields,total_goggles,total_bp_cuffs,total_wipers,total_ventilators
0,0,Statewide Unallocated,AL,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1001,Autauga County,AL,1,155,7,8.0,27.0,26.0,26.0,7.0,2.0,2.0,0.0
2,1003,Baldwin County,AL,1,620,28,31.0,107.0,104.0,104.0,29.0,7.0,7.0,1.0
3,1015,Calhoun County,AL,1,310,14,16.0,53.0,52.0,52.0,15.0,3.0,3.0,0.0
4,1017,Chambers County,AL,1,775,35,39.0,133.0,130.0,130.0,36.0,8.0,8.0,1.0


In [39]:
gpd_df = gpd.read_file(pop_geo)
gpd_df.head()

,GEO_ID,STATE_FIPS,County Name,LSAD,CENSUSAREA,State,COUNTY_FIPS,Population,Population (20+),Population (65+),geometry
0,0500000US01001,01,Autauga,County,594.436,AL,01001,55601.0,41177.0,8653.0,"POLYGON ((-86.49677 32.34444, -86.71790 32.402..."
1,0500000US01009,01,Blount,County,644.776,AL,01009,57840.0,43175.0,10548.0,"POLYGON ((-86.57780 33.76532, -86.75914 33.840..."
2,0500000US01017,01,Chambers,County,596.531,AL,01017,33615.0,25934.0,6562.0,"POLYGON ((-85.18413 32.87053, -85.12342 32.772..."
3,0500000US01021,01,Chilton,County,692.854,AL,01021,44153.0,32683.0,7411.0,"POLYGON ((-86.51734 33.02057, -86.51596 32.929..."
4,0500000US01033,01,Colbert,County,592.619,AL,01033,54762.0,42196.0,10914.0,"POLYGON ((-88.13999 34.58170, -88.13925 34.587..."


In [40]:
len(gpd_df), len(total_ventilators)

(3221, 1403)

In [41]:
gpd_df['COUNTY_FIPS'].dtype, total_ppes_df['COUNTY_FIPS'].dtype

(dtype('O'), dtype('int64'))

In [42]:
gpd_df['COUNTY_FIPS'] =gpd_df['COUNTY_FIPS'].astype('int64')

In [43]:
gpd_df_new = gpd_df

In [44]:
gpd_df_new = gpd_df_new.drop(columns = ['LSAD', 'CENSUSAREA', 'Population'])
gpd_df_new.head()

,GEO_ID,STATE_FIPS,County Name,State,COUNTY_FIPS,Population (20+),Population (65+),geometry
0,0500000US01001,01,Autauga,AL,1001,41177.0,8653.0,"POLYGON ((-86.49677 32.34444, -86.71790 32.402..."
1,0500000US01009,01,Blount,AL,1009,43175.0,10548.0,"POLYGON ((-86.57780 33.76532, -86.75914 33.840..."
2,0500000US01017,01,Chambers,AL,1017,25934.0,6562.0,"POLYGON ((-85.18413 32.87053, -85.12342 32.772..."
3,0500000US01021,01,Chilton,AL,1021,32683.0,7411.0,"POLYGON ((-86.51734 33.02057, -86.51596 32.929..."
4,0500000US01033,01,Colbert,AL,1033,42196.0,10914.0,"POLYGON ((-88.13999 34.58170, -88.13925 34.587..."


In [45]:
total_ppes_geo = gpd_df_new.merge(total_ppes_df, on='COUNTY_FIPS', how='left')

,GEO_ID,STATE_FIPS,County Name_x,State_x,COUNTY_FIPS,Population (20+),Population (65+),geometry,County Name_y,State_y,...,total_ppes,testkits,total_droplet_masks,total_n95s,total_gowns,total_faceshields,total_goggles,total_bp_cuffs,total_wipers,total_ventilators
0,0500000US01001,01,Autauga,AL,1001,41177.0,8653.0,"POLYGON ((-86.49677 32.34444, -86.71790 32.402...",Autauga County,AL,...,155.0,7.0,8.0,27.0,26.0,26.0,7.0,2.0,2.0,0.0
1,0500000US01009,01,Blount,AL,1009,43175.0,10548.0,"POLYGON ((-86.57780 33.76532, -86.75914 33.840...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0500000US01017,01,Chambers,AL,1017,25934.0,6562.0,"POLYGON ((-85.18413 32.87053, -85.12342 32.772...",Chambers County,AL,...,775.0,35.0,39.0,133.0,130.0,130.0,36.0,8.0,8.0,1.0
3,0500000US01021,01,Chilton,AL,1021,32683.0,7411.0,"POLYGON ((-86.51734 33.02057, -86.51596 32.929...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0500000US01033,01,Colbert,AL,1033,42196.0,10914.0,"POLYGON ((-88.13999 34.58170, -88.13925 34.587...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
total_ppes_geo = total_ppes_geo.drop(columns=['County Name_y', 'State_y', 'stateFIPS'])
total_ppes_geo = total_ppes_geo.rename(columns={'County Name_x': 'County Name',
                                               'State_x': 'State'})
total_ppes_geo.head()

,GEO_ID,STATE_FIPS,County Name,State,COUNTY_FIPS,Population (20+),Population (65+),geometry,total_ppes,testkits,total_droplet_masks,total_n95s,total_gowns,total_faceshields,total_goggles,total_bp_cuffs,total_wipers,total_ventilators
0,0500000US01001,01,Autauga,AL,1001,41177.0,8653.0,"POLYGON ((-86.49677 32.34444, -86.71790 32.402...",155.0,7.0,8.0,27.0,26.0,26.0,7.0,2.0,2.0,0.0
1,0500000US01009,01,Blount,AL,1009,43175.0,10548.0,"POLYGON ((-86.57780 33.76532, -86.75914 33.840...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0500000US01017,01,Chambers,AL,1017,25934.0,6562.0,"POLYGON ((-85.18413 32.87053, -85.12342 32.772...",775.0,35.0,39.0,133.0,130.0,130.0,36.0,8.0,8.0,1.0
3,0500000US01021,01,Chilton,AL,1021,32683.0,7411.0,"POLYGON ((-86.51734 33.02057, -86.51596 32.929...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0500000US01033,01,Colbert,AL,1033,42196.0,10914.0,"POLYGON ((-88.13999 34.58170, -88.13925 34.587...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
total_ppes_geo.to_file("../../../ppe_needs_county_level.geojson", driver='GeoJSON')